# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235239862387                   -0.50    8.0
  2   -7.250246804955       -1.82       -1.40    1.0
  3   -7.251304420606       -2.98       -2.12    3.0
  4   -7.251262459570   +   -4.38       -2.19    3.0
  5   -7.251328467426       -4.18       -2.65    2.0
  6   -7.251337752763       -5.03       -3.10    2.0
  7   -7.251338768122       -5.99       -3.94    1.0
  8   -7.251338791371       -7.63       -4.11    4.0
  9   -7.251338798235       -8.16       -4.68    2.0
 10   -7.251338798649       -9.38       -5.16    3.0
 11   -7.251338798696      -10.33       -5.56    3.0
 12   -7.251338798702      -11.20       -5.82    2.0
 13   -7.251338798704      -11.61       -6.34    2.0
 14   -7.251338798705      -12.72       -7.01    3.0
 15   -7.251338798705      -13.97       -7.43    3.0
 16   -7.251338798705      -14.75       -7.83    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.02820621350682627
[ Info: Arnoldi iteration step 2: normres = 0.5712269784442489
[ Info: Arnoldi iteration step 3: normres = 0.4339485283513704
[ Info: Arnoldi iteration step 4: normres = 0.9947259080918336
[ Info: Arnoldi iteration step 5: normres = 0.29006605419070747
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.58e-01, 7.82e-02, 1.60e-01, 1.65e-01, 1.98e-02)
[ Info: Arnoldi iteration step 6: normres = 0.6250257183453204
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.34e-02, 1.34e-01, 5.76e-01, 1.28e-01, 1.21e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07674855501181592
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.54e-03, 3.38e-02, 2.63e-02, 2.80e-02, 4.92e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10467683799819423
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.59e-